In [ ]:
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import en_core_web_sm
nlp=en_core_web_sm.load()
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer


In [2]:
import plotly.figure_factory as ff
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn

from gensim.models import Word2Vec

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.



In [3]:
from google.colab import files
uploaded = files.upload()

Saving comment_on_plan.csv to comment_on_plan (1).csv


In [4]:
import pandas as pd
df=pd.read_csv('comment_on_plan.csv',encoding = "ISO-8859-1")


In [5]:
df.head(5)

,Comment
0,The vaccine plan according to liberals is in t...
1,Starting a pandemic with masks can only be wor...
2,Dont forget the Libs tossing out our PPE and ...
3,they threw out emergency stock of N95s last ye...
4,Conservatives want to take a vaccine that hasn...


In [6]:
tokens=nlp(''.join(str(df.Comment.tolist())))

NER

In [9]:
items = [x.text for x in tokens.ents]
Counter(items).most_common(20)

[('Trudeau', 249),
 ('Canada', 157),
 ('Canadians', 93),
 ('first', 62),
 ('China', 50),
 ('US', 45),
 ('JT', 36),
 ('UK', 33),
 ('Health Canada', 32),
 ('Justin', 32),
 ('Moderna', 32),
 ('Canadian', 31),
 ('Chinese', 29),
 ('one', 25),
 ("O'Toole", 24),
 ('Liberals', 24),
 ('today', 18),
 ('LPC', 15),
 ('Trump', 15),
 ('Conservatives', 13)]

In [10]:
person_lst=[]
for ent in tokens.ents:
  if ent.label_=='PERSON':
    person_lst.append(ent.text)

person_counts=Counter(person_lst).most_common(20)
df_person=pd.DataFrame(person_counts,columns=['text','count'])


In [11]:
df_person.head(10)

,text,count
0,Trudeau,248
1,Moderna,17
2,Pfizer,13
3,Vaccine,11
4,Trump,9
5,Justin Trudeau,6
6,Lib,4
7,Cons,4
8,Tam,4
9,Patty,4


In [12]:
norp_lst=[]
for ent in tokens.ents:
  if ent.label_=='NORP':
    norp_lst.append(ent.text)
  
norp_counts=Counter(norp_lst).most_common(20)
df_norp=pd.DataFrame(norp_counts,columns=['text','count'])


In [13]:
df_norp.head(10)

,text,count
0,Canadians,93
1,Canadian,31
2,Chinese,29
3,Liberals,24
4,Conservatives,13
5,Americans,6
6,Feds,5
7,Russian,4
8,Conservative,3
9,Brits,3


In [14]:
org_list=[]
for ent in tokens.ents:
    if ent.label_ == 'ORG':
        org_list.append(ent.text)
        
org_counts = Counter(org_list).most_common(20)
df_org = pd.DataFrame(org_counts, columns =['text', 'count'])

In [15]:
df_org.head(10)

,text,count
0,Health Canada,32
1,Justin,32
2,O'Toole,24
3,LPC,15
4,CBC,12
5,NDP,8
6,RNA,7
7,UN,7
8,Con,7
9,CanSino,6


In [16]:
gpe_list = []
for ent in tokens.ents:
    if ent.label_ == 'GPE':
        gpe_list.append(ent.text)
        
gpe_counts = Counter(gpe_list).most_common(20)
df_gpe = pd.DataFrame(gpe_counts, columns =['text', 'count'])

In [17]:
df_gpe.head(10)

,text,count
0,Canada,157
1,China,50
2,US,45
3,JT,36
4,UK,33
5,Moderna,15
6,USA,12
7,Ottawa,11
8,Germany,9
9,Alberta,7


In [20]:
df['polarity']=df['Comment'].map(lambda text:TextBlob(text).sentiment.polarity)
df['commnet_len']=df['Comment'].astype(str).apply(len)
df['word_count']=df['Comment'].apply(lambda x: len(str(x).split()))

In [21]:
df.head(5)

,Comment,polarity,commnet_len,word_count
0,The vaccine plan according to liberals is in t...,0.258929,275,54
1,Starting a pandemic with masks can only be wor...,-0.031250,290,53
2,Dont forget the Libs tossing out our PPE and ...,-0.250000,86,17
3,they threw out emergency stock of N95s last ye...,0.000000,90,15
4,Conservatives want to take a vaccine that hasn...,0.000000,109,19


In [27]:
print('5 random comments with the highest positive sentiment polarity: \n')
cl=df.loc[df.polarity==1 , ['Comment']].sample(5).values
for c in cl:
  print(c[0])

5 random comments with the highest positive sentiment polarity: 

This fall update is going to be a confidence vote. The Cons greatest fear might be realized. An election where they would lose seats.
what's the rush? Take your Time, Wait until China has mastered the Vaccine - then we can millions and millions of doses from them to help their economy ...Why?....Because they are Awesome and deserve it
JT doing an excellent job.
He is PM because he is the best choice to protect Canadians from suffering the pain and shame of a Conservative government.
prorogued parliament is the best they can do


In [26]:
print('5 random comments with the most neutral sentiment(zero) polarity: \n')
cl=df.loc[df.polarity==0,['Comment']].sample(5).values
for c in cl:
  print(c[0])

5 random comments with the most neutral sentiment(zero) polarity: 

What WAS that drivel O'Toole evacuated yesterday about Canada putting all its vaccine eggs in China's basket. Doesn't that man read anything at all? The reformer/cons suffer from a lack of leadership .
If the vaccine works then anyone who takes it shouldn't care if others don't.
only in the movies.
Libs still adding contaminated liquid to their cornflakes for breakfast....??
like a math teacher getting a masters in basket weaving.


In [25]:
df.polarity.min()

-1.0

In [28]:
df.loc[df.polarity <= -0.7]

,Comment,polarity,commnet_len,word_count
97,Too bad the MSM prints snippets of O'Toole's y...,-0.75,152,28
387,I don't know about you guys but my worst fear ...,-1.00,141,24
620,An awful lot of faith is being given to an unp...,-0.75,117,22
1007,Trudeau destroyed the relationship and your bl...,-0.70,66,10
1109,The con base will vote Con every election when...,-0.80,76,14


In [29]:
print('5 comments with the most negative polarity: \n')
cl = df.loc[df.polarity <= -0.7, ['Comment']].sample(5).values
for c in cl:
    print(c[0])

5 comments with the most negative polarity: 

The con base will vote Con every election when these sorts of things happen.
An awful lot of faith is being given to an unproven vaccine. What if giving this to our frontline workers goes wrong?
Too bad the MSM prints snippets of O'Toole's yapping. I guess they have to do it for the sake of the elusive 'balance', but it's really, really annoying
I don't know about you guys but my worst fear is the Canadian government brainwashing and poisoning us by giving us rushed COVID-19 vaccines.
Trudeau destroyed the relationship and your blame game is pathetic


In [31]:
import re
def preprocessor(text):
    
    text = re.sub('<[^>]*>', '', text)
    text = re.sub(r'[^\w\s]','', text)
    text = text.lower()
    return text
    
df['Comment'] = df['Comment'].apply(preprocessor)

In [32]:
df.polarity.describe()

count    1362.000000
mean        0.073778
std         0.232625
min        -1.000000
25%         0.000000
50%         0.000000
75%         0.178571
max         1.000000
Name: polarity, dtype: float64

In [33]:
fig = ff.create_distplot([df['polarity'].tolist()], group_labels=['Comment Polarity Distribution plot'])
fig.show()

In [34]:

fig = ff.create_distplot([df['word_count'].tolist()], group_labels=['Word count Distribution plot'])
fig.show()


Most comments were short, less than 30 words, few comments can get as long as over 200 words

In [38]:
import spacy

nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) 

def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

df["comment_lemmatize"] =  df.apply(lambda x: lemmatizer(x['Comment']), axis=1)
df['comment_lemmatize'] = df['comment_lemmatize'].str.replace('-PRON-', '')

In [43]:

import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['comment_lemmatize'], 20)
df1 = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

fig = px.bar(df1, x='unigram', y='count')
fig.show()

In [44]:
def get_top_n_bigram(corpus, n=None):

    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['Comment'], 10)
df2 = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

fig = px.bar(df2, x='bigram', y='count')
fig.show()


a large number of people imitated Trudeau's response on vaccine roll out plan - "as quickly as possible", and it seemed they were skeptical on whatever "Trudeau says". This in align with a recent poll, finds that[ Most Canadians believe government is 'making it up as they go' with COVID vaccine.](https://nationalpost.com/news/canada/most-canadians-believe-government-is-making-it-up-as-they-go-with-covid-poll-finds)

In [45]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3,                        # minimum required occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             max_features=5000,             # max number of unique words. Build a vocabulary that only consider the top max_features ordered by term frequency across the corpus
                            )

data_vectorized = vectorizer.fit_transform(df['comment_lemmatize'])

lda_model = LatentDirichletAllocation(n_components=15, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

In [46]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
14     114.344879  114.308296       1        1  34.745268
2      105.821045  -93.749626       2        1  11.244735
11    -138.334473   95.320763       3        1  10.221830
8       -7.436697  111.259697       4        1   6.714709
12    -207.550629 -105.472603       5        1   5.833843
10      52.817650   14.603765       6        1   5.167888
3      168.010849    6.806950       7        1   4.183778
6      -94.169601 -157.789551       8        1   4.148478
0     -122.849174  -39.059418       9        1   3.860394
4      -94.894264  195.408295      10        1   3.668830
9       24.927401 -180.464874      11        1   3.118218
7      -53.413467   26.832108      12        1   2.513180
5      -12.255850  -72.093201      13        1   1.882707
1     -224.217957   28.070581      14        1   1.449797
13      36.154255  209.427582      15        1   1.246346, topic_info=           Term        Freq       Total Category  logprob  loglift
1016    vaccine  470.000000  470.000000  Default  30.0000  30.0000
532     liberal   91.000000   91.000000  Default  29.0000  29.0000
131      canada  188.000000  188.000000  Default  28.0000  28.0000
764     quickly  209.000000  209.000000  Default  27.0000  27.0000
711    possible  216.000000  216.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
410        good    0.082308   69.420800  Topic15  -7.5691  -2.3525
904     special    0.069009    4.311978  Topic15  -7.7453   0.2500
85    basically    0.069013    4.557945  Topic15  -7.7452   0.1946
818       right    0.070665   40.081195  Topic15  -7.7216  -1.9558
505        just    0.070775  116.696914  Topic15  -7.7200  -3.0229

[715 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0        12  0.889468   100
1        14  0.766903   1st
3         1  0.515764  2021
3        10  0.421989  2021
4         3  0.870804  2022
...     ...       ...   ...
1065      1  0.488546   yes
1065      3  0.183205   yes
1065      6  0.061068   yes
1065     12  0.183205   yes
1065     14  0.122136   yes

[1049 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 3, 12, 9, 13, 11, 4, 7, 1, 5, 10, 8, 6, 2, 14])